In [17]:
from bs4 import BeautifulSoup
import requests
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords

import smtplib  # To send the email

# To create the email body
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import datetime

In [18]:
now = datetime.datetime.now()

In [19]:
content = ''

In [20]:
def extract_website_data(url):
    response = requests.get(url)
    content = response.content
    soup = soup = BeautifulSoup(content, 'html.parser') 
    links = soup.find_all('h2',{'class':'c-entry-box--compact__title'})
    return soup, links

In [21]:
def get_only_text(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
    title = ' '.join(soup.title.stripped_strings)
    return title, text

In [51]:
list_of_links = []

def generate_content(links):
    cnt = ''
    for i in list(range(len(links))):
        link = links[i].a.get('href')
        list_of_links.append(link)
        page_content = get_only_text(link)
        summary = summarize(repr(page_content[1]), ratio=0.05)
        cnt += ('<br>' + page_content[0] + link + '\n' + summary + '\n' + '\n' + '<br>')
    return cnt

In [52]:
soup, links = extract_website_data("https://www.vox.com/")
content = generate_content(links)
content += ('<br>--------------<br>') # Mark the end of the email
content += ('<br><br>End of message')

In [53]:
print(content)

<br>Why is Putin attacking Ukraine? He told us. - Voxhttps://www.vox.com/policy-and-politics/2022/2/23/22945781/russia-ukraine-putin-speech-transcript-february-22
The speech is consistent with a body of statements from the Russian president going back years, ranging from a 5,000-word essay on Ukrainian history published last year to a 2005 speech declaring that “the collapse of the Soviet Union was a major geopolitical disaster [in which] tens of millions of our co-citizens and compatriots found themselves outside Russian territory.” That Putin truly believes in aggressive Russian nationalism does not make a larger invasion inevitable.
“Since time immemorial, the people living in the south-west of what has historically been Russian land have called themselves Russians.” What we now call Ukraine, he says, “was entirely created by Russia or, to be more precise, by Bolshevik Communist Russia.” In this questionable narrative, a trio of early Soviet leaders — Lenin, Stalin, and Khrushchev —

In [55]:
# Sending the email...
print('Composing email...')
# First create the parameters required to send an email
SERVER = 'smtp.gmail.com' # Your smtp server
PORT = 587 # For gmail it is 587
FROM = '********' # The senders email address
TO = '*********' # The receivers email
PASS = '*********'

# We create a empty object to create the body of the email using MIMEMultipart()
msg = MIMEMultipart()

# We create a dynamic subject to the email using datetime
msg['Subject'] = 'Top News Stories HN [Automated Email]' + '' + str(now.day) + '_'+ str(now.month) + '_'+ str(now.year)
msg['From'] = FROM
msg['To'] = TO

# To make the email look easthetically more appealing we use the html format
msg.attach(MIMEText(content, 'html'))

# Authenticating the email
print('Initializing the server')

# Assign the server and the port
server = smtplib.SMTP(SERVER, PORT)
# If you want to see if there is any error like failing to connect to the server put "1" in the debug level
server.set_debuglevel(1)
# Initiate the server
server.ehlo()
# start the connection
server.starttls()
# logging using the id and pass
server.login(FROM, PASS)
# Send the mail
server.sendmail(FROM, TO, msg.as_string())
print('Email sent...')
server.quit()

Composing email...
Initializing the server


send: 'ehlo LAPTOP-IGFO2I4P.gha.chartermi.net\r\n'
reply: b'250-smtp.gmail.com at your service, [2600:6c4a:727f:2500:d922:7eac:ee8c:533]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [2600:6c4a:727f:2500:d922:7eac:ee8c:533]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo LAPTOP-IGFO2I4P.gha.chartermi.net\r\n'
reply: b'250-smtp.gmail.com at your service, [2600:6c4a:727f:2500:d922:7eac:ee8c:533]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELININ

Email sent...


reply: b'250 2.0.0 OK  1645646362 i15sm299188oad.17 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1645646362 i15sm299188oad.17 - gsmtp'
data: (250, b'2.0.0 OK  1645646362 i15sm299188oad.17 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection i15sm299188oad.17 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection i15sm299188oad.17 - gsmtp'


(221, b'2.0.0 closing connection i15sm299188oad.17 - gsmtp')